Models

In [20]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import matplotlib.pyplot as plt

In [10]:
# Linear model
scaled_data = pd.read_csv('info_data.csv')
data_info = pd.read_csv('info_datav2.csv')
originaldata = pd.read_csv('../Data/Venta_Consumidor_Producto_UPC_PrevVTA_2018.csv')

In [18]:
traindataset, testdataset = train_test_split(data_info, test_size=0.3, random_state=42)
x_train = traindataset.loc[:, traindataset.columns != 'IMP_VENTA_NETO_EUR']
y_train = traindataset.loc[:, traindataset.columns == 'IMP_VENTA_NETO_EUR']
x_train = x_train.drop(columns=['EDAD_RANGO_COMPRA'])

In [19]:
lm = LinearRegression()
model = lm.fit(x_train, y_train)

In [ ]:
#mean_squared_error(y_true, y_pred)